In [1]:
%%capture
!pip3 install ijson
!pip3 install json
!pip3 install send2trash
from google.colab import drive, files
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········


In [2]:
import sys
import matplotlib.pyplot as plt
import keras
import tensorflow
import pandas as pd
from keras.layers import *
from pathlib import Path

np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%.3g" % x))
GOOGLE_COLAB = "google.colab" in sys.modules
if GOOGLE_COLAB:
    sys.path.append("./gdrive/My Drive/Colab Notebooks/solar_flares")
    plt.style.use("default")
    config = tensorflow.ConfigProto(device_count={"GPU": 1})
    keras.backend.set_session(tensorflow.Session(config=config))
    from lstm_fcn.lstmfcn_model import *
    from reading_data import load_npz_file


else:
    config = tensorflow.ConfigProto(intra_op_parallelism_threads=8, inter_op_parallelism_threads=8)
    keras.backend.tensorflow_backend.set_session(tensorflow.Session(config=config))
    from lstm_fcn.lstmfcn_model import *
    from reading_data import load_npz_file





Using TensorFlow backend.


In [5]:
if __name__ == '__main__':
    ROOTDIR = "/Users/mag/Google Drive/Colab Notebooks/solar_flares/"
    DATADIR = "input/npz"
    OUTDIR = "output"
    DATANAME = "lstm_fcn"
    TAG = "spatialdropout"

    TRAIN = True
    NEW_MODEL = True

    TEST = True

    EPOCHS = 10
    PERCENTAGE = 100
    BATCH_SIZE = 128
    CALLBACK_TIMEOUT = 15
    N_TIMESTEPS = 60  # Change if Variable length
    INCLUDE_E = True
    INCLUDE_S = True
    SCALER = sklearn.preprocessing.maxabs_scale
    STATIONARY = False
    if TEST:
        model_name = "{}_test_model.h5".format(DATANAME)
    else:
        model_name = "{}_best_model.h5".format(DATANAME)

    if GOOGLE_COLAB:
        ROOTDIR = "./gdrive/My Drive/Colab Notebooks" + str(ROOTDIR).split("Colab Notebooks")[-1]

    rootdir = Path(ROOTDIR)
    datadir = rootdir.joinpath(DATADIR)
    outdir = rootdir.joinpath(OUTDIR)



    if TEST:
        X, y = load_npz_file(datadir / 'fold3Training.npz')
    else:
        X1, y1 = load_npz_file(datadir / 'fold1Training.npz')
        X2, y2 = load_npz_file(datadir / 'fold2Training.npz')
        X3, y3 = load_npz_file(datadir / 'fold3Training.npz')

        X = np.concatenate([X1, X2, X3])
        y = np.concatenate([y1, y2, y3])
        
    print("Data loaded.")
    
    y = keras.utils.to_categorical(y, num_classes=2)

    # preprocess x

    X = sklearn.preprocessing.StandardScaler().fit_transform(X.reshape((X.shape[0], X.shape[1]*X.shape[2]))).reshape(X.shape)


    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
        X, y)

    del X
    del y
    
    
    model = get_model(
        n_features=X_train.shape[-1],
        train=TRAIN,
        new_model=NEW_MODEL,
        model_name=model_name,
        model_path=outdir,
        google_colab=GOOGLE_COLAB,
    )

    # if TAG is not None:
    #     DATANAME += "_" + TAG
    #     model_name = model_name.replace("best_model", TAG + "_best_model")

    if TRAIN:
        # callbacks = generate_callbacks(patience=CALLBACK_TIMEOUT, outdir=outdir, name=DATANAME)
        callbacks = Metrics()
        model.fit(
            x=X_train,
            y=y_train,
            validation_data=(X_test, y_test),
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=callbacks,
        )
        # lib.plotting.plot_losses(logpath=outdir, outdir=outdir, name=DATANAME)

        if GOOGLE_COLAB:
            print("Converted model from GPU to CPU-compatible")
            cpu_model = create_model(google_colab=False, n_features=X_train.shape[-1])
            gpu_model_to_cpu(
                trained_gpu_model=model, untrained_cpu_model=cpu_model, outdir=outdir, modelname=model_name
            )

    print("Evaluating...")
    y_pred = model.predict(X_test)



    #for yp, yt in zip(np.argmax(y_pred, axis=1), np.argmax(y_test, axis=1)):
    #    print('T:{} P:{}'.format(yt,yp))
    plot_confusion_matrices(
        y_target=np.argmax(y_test,axis=1), y_pred=np.argmax(y_pred,axis=1), y_is_binary=True, outdir=outdir, name=DATANAME
    )
    print("F1 score: {}".format(f1(y_test, y_pred)))
    # 
    # df = pd.DataFrame({'Id':ids,'ClassLabel':np.argmax(y_pred,axis=1)})
    # df.to_csv(outdir/'submission.csv')

Data loaded.
Created new model.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, 25)     0                                            
__________________________________________________________________________________________________
conv1d_10 (Conv1D)              (None, None, 128)    25728       input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_10 (BatchNo (None, None, 128)    512         conv1d_10[0][0]                  
__________________________________________________________________________________________________
activation_10 (Activation)      (None, None, 128)    0           batch_normalization_10[0][0]     
_____________________________________________________________________________

./gdrive/My Drive/Colab Notebooks/solar_flares/lstm_fcn/lstmfcn_model.py:122: RuntimeWarning: Didn't trash file (probably because of Google Drive)
  warn("Didn't trash file (probably because of Google Drive)", RuntimeWarning)


Evaluating...
F1 score: Tensor("Mean_1:0", shape=(), dtype=float32)


/usr/local/lib/python3.6/dist-packages/mlxtend/plotting/plot_confusion_matrix.py:59: RuntimeWarning: invalid value encountered in true_divide
  normed_conf_mat = conf_mat.astype('float') / total_samples
